<br><br><br>
<center> <h3 style="color:blue">========================================================</h3> </center>
<center> <h3 style="color:green">In the Name of Allah, the Most Beneficent, the Most Merciful</h3> </center>
<center> <h3 style="color:green">AI 2024 Online Summer Internship</h3> </center>
<center> <h3 style="color:green">Make Your Place in Top 20% AI Engineers of the World</h3> </center>
<center> <h3 style="color:blue">========================================================</h3> </center>
<br><br><br>

        *----------------------------- AUTHOR_DETAILS -------------------------------*
        |                                                                            |
        |        Project Title  = Multi-Label Course Classification                  |
        |                                                                            |
        |        Author         = H. M. Adil Javaid                                  |
        |                                                                            |
        |        Copyright      = Copyright (C) 2024 H. M. Adil Javaid               |
        |                                                                            |
        |        License        = Public Domain                                      |
        |                                                                            |
        |        Version        = 1.0                                                |
        |                                                                            |
        *----------------------------------------------------------------------------*

<br><br><br>
<center> <h2 style="color:green">-------------------- PROJECT PURPOSE --------------------</h2> </center>
<br>
<center><h3>
The main purpose of this project is to demonstrate how the Multi-label Courses Classification problem can be treated as a Supervised Machine Learning problem using Python and the Mistral-7b Model.
</h3></center>
<br>
<center><h3>For this purpose, In Sha Allah, we will execute the Zero Shot Approach.</h3></center>
<br>
<center> <h2 style="color:green">-------------------------------------------------------------------------</h2> </center>
<br><br><br>


### In Sha Allah, we will follow the following steps to execute the Zero Shot Approach

#### Step 1: Installing Required Libraries for Language Models and LangChain Framework

#### Step 2: Import Libraries
    

#### Step 3: Initializing and Configuring the Mistral-7b Model with Quantization for Text Generation


#### Step 4: Creating a Hugging Face Pipeline for Language Model Integration 

#### Step 5: Defining Output Labels

#### Step 6: Extracting and Identifying Output Labels from Text Data

#### Step 7: Load Test Data 

#### Step 8: Generating Output Predictions from Text Data Using Language Model Queries

#### Step 9: DataFrames for Predicted and Actual Output Labels 

    Step 9.1: Convert into DataFrame

    Step 9.2: Comparing Actual and Predicted Course Labels for Sample Data


#### Step 10: Evaluation Metrics for Output Label Predictions

# Step 1: Installing Required Libraries for Language Models and LangChain Framework

In [ ]:
!pip install transformers --quiet
!pip install accelerate --quiet
!pip install bitsandbytes --quiet
!pip install langchain --quiet

^C
ERROR: Operation cancelled by user


In [7]:
pip install langchain-community langchain-core --quiet

Note: you may need to restart the kernel to use updated packages.


# Step 2: Import Libraries

In [11]:
# General packages
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from textwrap import fill
from IPython.display import Markdown, display # for formating Python display folowing markdown language
import warnings
warnings.filterwarnings('ignore') # avoid warning messages importing packages

In [12]:
# Mistral and LangChain packages (prompt engineering)
import torch
from langchain import PromptTemplate, HuggingFacePipeline
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

2024-08-20 16:38:28.406074: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 16:38:28.406182: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 16:38:28.536871: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Step 3: Initializing and Configuring the Mistral-7b Model with Quantization for Text Generation

In [13]:
# Model version of Mistral
MODEL_NAME = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

# Quantization is a technique used to reduce the memory and computation requirements 
# of deep learning models, typically by using fewer bits, 4 bits
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Initialization of a tokenizer for the Mistral-7b model, 
# necessary to preprocess text data for input
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Initialization of the pre-trained language Mistral-7b
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

# Configuration of some generation-related settings
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 20 # maximum number of new tokens that can be generated by the model
generation_config.temperature = 0.7 # randomness of the generated tex
generation_config.top_p = 0.95 # diversity of the generated text
generation_config.do_sample = True # sampling during the generation process
generation_config.repetition_penalty = 1.15 # the degree to which the model should avoid repeating tokens in the generated text

# A pipeline is an object that works as an API for calling the model
# The pipeline is made of (1) the tokenizer instance, the model instance, and
# some post-procesing settings. Here, it's configured to return full-text outputs
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Step 4: Creating a Hugging Face Pipeline for Language Model Integration

In [14]:
# HuggingFace pipeline
llm = HuggingFacePipeline(pipeline=pipe)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


# Step 5: Defining Output Labels

In [16]:
import re
# Define the courses labels
courses_labels = ["Computer", "Physics", "Mathematics", "Statistics", "Quantitative Biology", "Quantitative Finance"]

# Step 6: Extracting and Identifying Output Labels from Text Data

In [22]:
def extract_courses(text):
    """Extracts text outside [INST] and [/INST] tags and matches against courses_labels."""
    last_inst_index = text.rfind("[/INST]")
    if last_inst_index != -1:
        my_text = text[last_inst_index + len("[/INST]"):].strip().lower()
        # Use regex to replace unwanted characters and split by whitespace
        clean_text = re.sub(r"[^\w\s]", " ", my_text)
        detected_courses = clean_text.split()

        # Debugging output (optional)
        # print('Detected Courses:', detected_courses)

        # Return a list indicating which courses are detected
        return [1 if course.lower() in detected_courses else 0 for course in courses_labels]
    else:
        return [0] * len(courses_labels)


# Step 7: Load Test Data

In [30]:
import pandas as pd

# Load the dataset
data = pd.read_csv('/kaggle/input/abstract-classification/Test_Sample.csv')

# Create a copy of the dataset
data_t = data.copy()

# Extract the 'ABSTRACT' column as test texts
test_texts = data_t['ABSTRACT'].tolist()

# Drop 'ID', 'TITLE', and 'ABSTRACT' columns to extract the actual values for emotions
test_labels = data_t.drop(columns=['ID', 'TITLE', 'ABSTRACT']).values


# Step 8: Generating Output Predictions from Text Data Using Language Model Queries

In [31]:
predictions = []
for text in test_texts:
    #text = "i want to go out  of this house"
    # In English
    query = f"""[INST]you are an expert who need to assess and detect courses present in given text.
            please identify and list up to four prominent courses present in the provided English text. 
            please Only Consider the following emotion labels:
            Computer
            Physics
            Mathematics
            Statistics
            Quantitative Biology
            Quantitative Finance

            If no courses are detected or the text is unclear, label as ['neutral'].

            your answer should be in the form of a "Python List" containing only available courses labels.

            you will only respond with the label given. Do not include the word "label". Do not provide explanation of notes.

            Text: {text}
            Courses: 
            .[/INST] """
    result = llm(query)
    #display(Markdown(f"<b>{query.removeprefix('[INST]').removesuffix('[/INST]')}</b>"))
    #display(Markdown(f"<p>{result}</p>"))
    r_label = extract_courses(result)
    print("****", r_label)
    predictions.append(r_label)

**** [0, 0, 0, 0, 0, 0]
**** [0, 1, 1, 1, 0, 0]
**** [1, 0, 0, 0, 0, 0]
**** [1, 0, 0, 0, 0, 0]
**** [1, 1, 0, 0, 0, 0]
**** [1, 0, 0, 0, 0, 0]
**** [1, 1, 1, 0, 0, 0]
**** [1, 1, 1, 1, 0, 0]
**** [1, 0, 1, 0, 0, 0]
**** [1, 0, 0, 0, 0, 0]
**** [0, 1, 1, 0, 0, 0]
**** [0, 0, 0, 0, 0, 0]
**** [1, 0, 1, 1, 0, 0]
**** [1, 0, 1, 1, 0, 0]
**** [1, 1, 1, 1, 0, 0]
**** [1, 0, 0, 0, 0, 0]
**** [1, 1, 1, 1, 0, 0]
**** [1, 0, 1, 0, 0, 0]
**** [1, 1, 1, 1, 0, 0]
**** [0, 0, 0, 0, 0, 0]
**** [1, 0, 0, 0, 0, 0]
**** [0, 1, 1, 1, 0, 0]
**** [1, 0, 1, 1, 0, 0]


# Step 9: DataFrames for Predicted and Actual Output Labels

## Step 9.1: Convert into DataFrame

In [32]:
results_df = pd.DataFrame(predictions, columns=courses_labels)
print(results_df)
actual_df = pd.DataFrame(test_labels, columns=courses_labels)
print(actual_df)

    Computer  Physics  Mathematics  Statistics  Quantitative Biology  \
0          0        0            0           0                     0   
1          0        1            1           1                     0   
2          1        0            0           0                     0   
3          1        0            0           0                     0   
4          1        1            0           0                     0   
5          1        0            0           0                     0   
6          1        1            1           0                     0   
7          1        1            1           1                     0   
8          1        0            1           0                     0   
9          1        0            0           0                     0   
10         0        1            1           0                     0   
11         0        0            0           0                     0   
12         1        0            1           1                  

## Step 9.2: Comparing Actual and Predicted Course Labels for Sample Data

In [33]:
for i in range(20):
    print(test_labels[i],"\n",predictions[i], "\n\n")

[0 0 1 1 0 0] 
 [0, 0, 0, 0, 0, 0] 


[0 1 1 0 0 0] 
 [0, 1, 1, 1, 0, 0] 


[1 0 0 1 0 0] 
 [1, 0, 0, 0, 0, 0] 


[0 0 1 0 0 0] 
 [1, 0, 0, 0, 0, 0] 


[1 0 0 0 0 0] 
 [1, 1, 0, 0, 0, 0] 


[1 0 1 0 0 0] 
 [1, 0, 0, 0, 0, 0] 


[0 1 0 0 0 0] 
 [1, 1, 1, 0, 0, 0] 


[0 1 0 0 0 0] 
 [1, 1, 1, 1, 0, 0] 


[1 0 1 0 0 0] 
 [1, 0, 1, 0, 0, 0] 


[1 0 0 0 0 0] 
 [1, 0, 0, 0, 0, 0] 


[0 1 0 0 0 0] 
 [0, 1, 1, 0, 0, 0] 


[1 0 0 0 0 0] 
 [0, 0, 0, 0, 0, 0] 


[1 0 0 1 0 0] 
 [1, 0, 1, 1, 0, 0] 


[1 0 0 0 0 0] 
 [1, 0, 1, 1, 0, 0] 


[0 0 1 0 0 0] 
 [1, 1, 1, 1, 0, 0] 


[1 0 0 1 0 0] 
 [1, 0, 0, 0, 0, 0] 


[0 0 1 0 0 0] 
 [1, 1, 1, 1, 0, 0] 


[0 0 1 1 0 0] 
 [1, 0, 1, 0, 0, 0] 


[0 1 0 0 0 0] 
 [1, 1, 1, 1, 0, 0] 


[1 0 1 1 0 0] 
 [0, 0, 0, 0, 0, 0] 




# Step 10: Evaluation Metrics for Output Label Predictions

In [34]:
from sklearn.metrics import f1_score, precision_score, recall_score
# Compute evaluation metrics
f1_micro = f1_score(test_labels, predictions, average='micro')
f1_macro = f1_score(test_labels, predictions, average='macro')
precision_micro = precision_score(test_labels, predictions, average='micro')
precision_macro = precision_score(test_labels, predictions, average='macro')
recall_micro = recall_score(test_labels, predictions, average='micro')
recall_macro = recall_score(test_labels, predictions, average='macro')

# Print the evaluation metrics
print(f"F1 Micro: {f1_micro}")
print(f"F1 Macro: {f1_macro}")
print(f"Precision Micro: {precision_micro}")
print(f"Precision Macro: {precision_macro}")
print(f"Recall Micro: {recall_micro}")
print(f"Recall Macro: {recall_macro}")

F1 Micro: 0.5542168674698795
F1 Macro: 0.36076604554865427
Precision Micro: 0.4791666666666667
Precision Macro: 0.31330651918887215
Recall Micro: 0.6571428571428571
Recall Macro: 0.4268398268398268


<br><br><br>
<center> <h3 style="color:blue">==========================================================</h3> </center>
<center> <h3 style="color:green">JAZAK ALLAH KHAIR</h3> 
<center> <h3 style="color:blue">==========================================================</h3> </center>
<br><br><br>